In [ ]:
from binance import Client
from datetime import datetime, timedelta
client = Client(api_key="jM9zEeTTzT7R7QGlNrh5tUpOc8l71hOxu7CewCL4M0NvbBp3ilgmEwMOOjJfCRVB",api_secret="iZxFmlz5bvROsHQPZTP5hlJtsxYjOu8wocOlLWzIDdhMzEQu7KyyHB2DseOYQ66s",testnet=True)

import os
from google.cloud.sql.connector import Connector
from sqlalchemy import create_engine, text
import pg8000.native
import pandas as pd
import numpy as np
import ta
from time import sleep

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'C:/Users/tinma/OneDrive/Escritorio/application_default_credentials.json'

connector = Connector()

def getconn() -> pg8000.native.Connection:
    conn: pg8000.native.Connection = connector.connect(
        "semiotic-pact-388619:us-central1:tradingbot",
        "pg8000",
        user="postgres",
        password="GuruSat.3",
        db="criptos"
    )
    return conn

connection = create_engine('postgresql+pg8000://',creator=getconn)

def stochastic_oscillator(df,n=90,m=17,lags=16):
    df['%K'] = pd.Series(dtype=float)
    for i in range(n,len(df)):
        df.loc[i-1,['%K']] = ((df.Precio[i-1] - min(df.Precio[i-n:i]))/(max(df.Precio[i-n:i]) - min(df.Precio[i-n:i]))) * 100

    df['%D'] = df['%K'].rolling(m).mean()

    dfc = pd.DataFrame()
    dfv = pd.DataFrame()
    dfx = pd.DataFrame()
    df['Señal_stoch2'] = pd.Series(dtype=int)
    df['Señal_venta_stoch1'] = pd.Series(dtype=int)
    df['Señal_compra_stoch1'] = pd.Series(dtype=int)

    for i in range(1,lags):
        maskc = (df['%K'].shift(i) < 20) & (df['%D'].shift(i) < 20)
        maskv = (df['%K'].shift(i) > 80) & (df['%D'].shift(i) > 80)
        dfc = pd.concat([dfc,maskc],axis=1)
        dfv = pd.concat([dfv,maskv],axis=1)
    df['Señal_venta_stoch1'] = np.where(dfv.sum(axis=1),1,0)
    df['Señal_compra_stoch1'] = np.where(dfc.sum(axis=1),1,0)  

    mask =  df['%K'].between(20,80) & df['%D'].between(20,80)
    df['Señal_stoch2'] = np.where(pd.concat([dfx,mask]),1,0)


def RSI(df,n=90):
    dfx = pd.DataFrame()
    df['Señal_venta_RSI'] = pd.Series(dtype=int)
    df['Señal_compra_RSI'] = pd.Series(dtype=int)
    df['RSI'] = ta.momentum.rsi(df.Precio,window=n)
    mask = df['RSI'] > 50
    df['Señal_compra_RSI'] = np.where(pd.concat([dfx,mask]),1,0)
    mask = df['RSI'] < 50
    df['Señal_venta_RSI'] = np.where(pd.concat([dfx,mask]),1,0)


def MACD(df):
    df['EMA12'] = df.Precio.ewm(span=12).mean()
    df['EMA26'] = df.Precio.ewm(span=26).mean()
    df['MACD'] = df.EMA12 - df.EMA26
    df['Signal'] = df.MACD.ewm(span=9).mean()
    df['Histogram'] = df.MACD - df.Signal
    df['Señal_venta_MACD'] = pd.Series(dtype=int)
    df['Señal_compra_MACD'] = pd.Series(dtype=int)
    for i in range(1,len(df)):
        if df.Histogram.iloc[i] > 0 and df.Histogram.iloc[i-1] < 0:
            df.loc[i,['Señal_compra_MACD']] = 1
        else:
            df.loc[i,['Señal_compra_MACD']] = 0

        if df.Histogram.iloc[i] < 0 and df.Histogram.iloc[i-1] > 0:
            df.loc[i,['Señal_venta_MACD']] = 1
        else:
            df.loc[i,['Señal_venta_MACD']] = 0


def Stochastich_RSI_MACD(df,n=90,m=17,lags=16):
    stochastic_oscillator(df,n=n,m=m,lags=lags)
    RSI(df,n=n)
    MACD(df)
    df.set_index('Tiempo')
    dfx = pd.DataFrame()
    df['Señal_Compra'] = pd.Series(dtype=int)
    df['Señal_Venta'] = pd.Series(dtype=int)
    mask_compra = (df['Señal_compra_stoch1'] == 1) & (df['Señal_stoch2'] == 1) & (df['Señal_compra_RSI'] == 1) & (df['Señal_compra_MACD'] == 1) 
    df['Señal_Compra'] = np.where(pd.concat([dfx,mask_compra]),1,0)
    mask_venta = (df['Señal_venta_stoch1'] == 1) & (df['Señal_stoch2'] == 1) & (df['Señal_venta_RSI'] == 1) & (df['Señal_venta_MACD'] == 1) 
    df['Señal_Venta'] = np.where(pd.concat([dfx,mask_venta]),1,0)
    df.dropna(inplace=True)
    df.set_index('Tiempo',inplace=True)

ConnectionError: HTTPSConnectionPool(host='testnet.binance.vision', port=443): Max retries exceeded with url: /api/v3/ping (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x0000025C17DE29E0>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))

['semiotic-pact-388619:us-central1:tradingbot']: An error occurred while performing refresh. Scheduling another refresh attempt immediately
Traceback (most recent call last):
  File "c:\Users\tinma\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connection.py", line 174, in _new_conn
    conn = connection.create_connection(
  File "c:\Users\tinma\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\util\connection.py", line 72, in create_connection
    for res in socket.getaddrinfo(host, port, family, socket.SOCK_STREAM):
  File "c:\Users\tinma\AppData\Local\Programs\Python\Python310\lib\socket.py", line 955, in getaddrinfo
    for res in _socket.getaddrinfo(host, port, family, type, proto, flags):
socket.gaierror: [Errno 11001] getaddrinfo failed

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\tinma\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py

In [223]:
client.get_account()

ConnectionError: HTTPSConnectionPool(host='testnet.binance.vision', port=443): Max retries exceeded with url: /api/v3/account?timestamp=1688411795792&signature=e32aac655b498836b7a3f94cd633b3e1c0f18683a9becbb5dcadc1fb7f2a3dfd (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x0000025C17954370>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))

In [227]:
day = datetime.utcnow() - timedelta(seconds=86400/12)

PAIR = 'ETHUSDT'

with connection.connect() as conn:
    df = conn.execute(text(""" SELECT * FROM "{PAIR}" WHERE "Tiempo" > '{day}' """.format(day=day,PAIR=PAIR))).mappings().all()
df = pd.DataFrame(df)[['Tiempo', 'Precio']]
df = df.sort_values(by='Tiempo').reset_index(drop=True)

Stochastich_RSI_MACD(df,n=90,m=17,lags=90)

Exception closing connection <pg8000.dbapi.Connection object at 0x0000025C10D7ED70>
Traceback (most recent call last):
  File "c:\Users\tinma\AppData\Local\Programs\Python\Python310\lib\site-packages\pg8000\core.py", line 147, in _flush
    sock.flush()
  File "c:\Users\tinma\AppData\Local\Programs\Python\Python310\lib\socket.py", line 723, in write
    return self._sock.send(b)
  File "c:\Users\tinma\AppData\Local\Programs\Python\Python310\lib\ssl.py", line 1206, in send
    return self._sslobj.write(data)
ssl.SSLEOFError: EOF occurred in violation of protocol (_ssl.c:2396)

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "c:\Users\tinma\AppData\Local\Programs\Python\Python310\lib\site-packages\sqlalchemy\engine\base.py", line 1900, in _execute_context
    self.dialect.do_execute(
  File "c:\Users\tinma\AppData\Local\Programs\Python\Python310\lib\site-packages\sqlalchemy\engine\default.py", line 736, in do_execute
    cur

InterfaceError: (pg8000.exceptions.InterfaceError) network error
[SQL:  SELECT * FROM "ETHUSDT" WHERE "Tiempo" > '2023-07-03 19:48:55.049706' ]
(Background on this error at: https://sqlalche.me/e/14/rvf5)

In [208]:
df

,Precio,%K,%D,Señal_stoch2,Señal_venta_stoch1,Señal_compra_stoch1,Señal_venta_RSI,Señal_compra_RSI,RSI,EMA12,EMA26,MACD,Signal,Histogram,Señal_venta_MACD,Señal_compra_MACD,Señal_Compra,Señal_Venta
Tiempo,,,,,,,,,,,,,,,,,,
2023-07-03 01:29:39.800,1941.50,77.876106,76.782926,1,0,0,0,1,57.652355,1941.547766,1941.365903,0.181862,0.173878,0.007984,0.0,0.0,0,0
2023-07-03 01:29:45.310,1941.59,81.858407,79.828214,0,0,0,0,1,58.415729,1941.554263,1941.382507,0.171756,0.173454,-0.001698,1.0,0.0,0,0
2023-07-03 01:29:45.907,1941.63,83.628319,82.977616,0,0,0,0,1,58.749926,1941.565915,1941.400845,0.165070,0.171777,-0.006707,0.0,0.0,0,0
2023-07-03 01:29:47.628,1941.63,83.628319,84.409162,0,1,0,0,1,58.749926,1941.575774,1941.417823,0.157951,0.169012,-0.011061,0.0,0.0,0,0
2023-07-03 01:29:50.612,1941.40,73.451327,84.174909,0,1,0,0,1,56.098976,1941.548732,1941.416503,0.132229,0.161655,-0.029426,0.0,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-07-03 03:24:57.887,1949.84,54.744526,59.493295,1,1,0,0,1,51.322145,1949.896642,1949.917669,-0.021027,-0.017050,-0.003977,0.0,0.0,0,0
2023-07-03 03:25:01.267,1949.85,55.474453,59.062927,1,1,0,0,1,51.431041,1949.889466,1949.912657,-0.023190,-0.018278,-0.004912,0.0,0.0,0,0
2023-07-03 03:25:01.411,1949.84,54.744526,58.589623,1,1,0,0,1,51.314956,1949.881856,1949.907275,-0.025418,-0.019706,-0.005712,0.0,0.0,0,0


In [210]:
buy = df[(df['Señal_Compra'] == 1)]['Precio']
sell = df[(df['Señal_Venta'] == 1)]['Precio']

In [228]:
path = 'C:/Users/tinma/OneDrive/Escritorio/tradingbot/trade_logs/testnet/Stoch_RSI_MACD/'
balances_p = []
PAIR = 'ETHUSDT'
pair = 'eth'
LOCALIDAD_SEÑAL_ENTRADA = 30 # SEGUNDOS
LOCALIDAD_SEÑAL_SALIDA = 30 # SEGUNDOS
UMBRAL_PERDIDA = - 5 # DÓLARES
open_order = False
balance_trade_anterior = 0
balance_trade_p = 0

while True:

######################## STREAMING Y ESTRATEGIA.

    day = datetime.utcnow() - timedelta(seconds=86400/12)

    with connection.connect() as conn:
        df =conn.execute(text(""" SELECT * FROM "{PAIR}" WHERE "Tiempo" > '{day}' """.format(day=day,PAIR=PAIR))).mappings().all()
    df = pd.DataFrame(df)[['Tiempo', 'Precio']]
    df = df.sort_values(by='Tiempo').reset_index()
    df.dropna(inplace=True)

    Stochastich_RSI_MACD(df,n=90,m=17,lags=90)

    buy = df[(df['Señal_Compra'] == 1)]['Precio']
    sell = df[(df['Señal_Venta'] == 1)]['Precio']

######################## MÉTODO DE ENTRADA POR MEDIO DE SEÑAL.

    if open_order == False:

        DIFF_SEÑAL_ENTRADA = (datetime.utcnow() - buy.index[-1]).total_seconds()

        if DIFF_SEÑAL_ENTRADA < LOCALIDAD_SEÑAL_ENTRADA:
            buy_order = client.create_order(
                        symbol=PAIR,
                        side=Client.SIDE_BUY,
                        type=Client.ORDER_TYPE_MARKET,
                        quantity=0.1,
                        recvWindow=60000
                    )
            open_order = True
            tiempo_entrada = datetime.utcnow()
            precio_entrada = float(buy_order['fills'][0]['price'])
            print(f'Se realizó una compra al precio: {precio_entrada}, en el momento: {tiempo_entrada}')

######################## MÉTODO DE SALIDA POR MEDIO DEL UMBRAL DE PÉRIDA.

    while open_order:

        if df.Precio[-1] - precio_entrada < UMBRAL_PERDIDA:
            sell_order = client.create_order(
                        symbol=PAIR,
                        side=Client.SIDE_SELL,
                        type=Client.ORDER_TYPE_MARKET,
                        quantity=0.1,
                        recvWindow=60000
                    )
            open_order = False
            tiempo_salida = datetime.utcnow()
            precio_salida = float(sell_order['fills'][0]['price'])
            balance_trade_p = (precio_salida - precio_entrada)*0.1
            balances_p.append(balance_trade_p)
      
######################## MÉTODO DE SALIDA POR MEDIO DE SEÑAL

        if open_order:

            DIFF_SEÑAL_SALIDA = (datetime.utcnow() - sell.index[-1]).total_seconds()

            if DIFF_SEÑAL_SALIDA < LOCALIDAD_SEÑAL_SALIDA: # and sell.index[-1] > buy.index[-1]:
                sell_order = client.create_order(
                            symbol=PAIR,
                            side=Client.SIDE_SELL,
                            type=Client.ORDER_TYPE_MARKET,
                            quantity=0.1,
                            recvWindow=60000
                        )
                open_order = False
                tiempo_salida = datetime.utcnow()
                precio_salida = float(sell_order['fills'][0]['price'])
                balance_trade_p = (precio_salida - precio_entrada)*0.1
                balances_p.append(balance_trade_p)

######################## LOGS

    if balance_trade_anterior != balance_trade_p:
        balance_trade_anterior = balance_trade_p
        if balance_trade_p>0:
            print(f'Se realizó una venta al precio: {precio_salida}, en el momento: {tiempo_salida}. Ganancia de: {balance_trade_p} dólares.')
            trade_log = pd.DataFrame([[precio_entrada, tiempo_entrada, precio_salida, tiempo_salida, (balance_trade_p)]],
                                columns=['Precio_Entrada','Tiempo_Entrada','Precio_Salida','Tiempo_Salida','Ganancia'])
            if os.path.exists(path+f'trades_ganados_{pair}.csv'):
                trade_log.to_csv(path+f'trades_ganados_{pair}.csv',
                                mode='a',
                                index=False,
                                header=False
                                )
            else:
                trade_log.to_csv(path+f'trades_ganados_{pair}.csv',
                                index=False,
                                header=True,
                                )
        elif balance_trade_p==0:
            print('No hubo pérdidas ni ganancias del trade')
        else:
            print(f'Se realizó una venta al precio: {precio_salida}, en el momento: {tiempo_salida}. Pérdida de: {balance_trade_p} dólares.')
            trade_log = pd.DataFrame([[precio_entrada, tiempo_entrada, precio_salida, tiempo_salida, (balance_trade_p)]],
                                columns=['Precio_Entrada','Tiempo_Entrada','Precio_Salida','Tiempo_Salida','Perdida'])
            if os.path.exists(path+f'trades_perdidos_{pair}.csv'):
                trade_log.to_csv(path+f'trades_perdidos_{pair}.csv',
                                mode='a',
                                index=False,
                                header=False
                                )
            else:
                trade_log.to_csv(path+f'trades_perdidos_{pair}.csv',
                                index=False,
                                header=True,
                                )           

Exception closing connection <pg8000.dbapi.Connection object at 0x0000025C0403BFD0>
Traceback (most recent call last):
  File "c:\Users\tinma\AppData\Local\Programs\Python\Python310\lib\site-packages\sqlalchemy\engine\base.py", line 1900, in _execute_context
    self.dialect.do_execute(
  File "c:\Users\tinma\AppData\Local\Programs\Python\Python310\lib\site-packages\sqlalchemy\engine\default.py", line 736, in do_execute
    cursor.execute(statement, parameters)
  File "c:\Users\tinma\AppData\Local\Programs\Python\Python310\lib\site-packages\pg8000\dbapi.py", line 456, in execute
    self._c.execute_simple("begin transaction")
  File "c:\Users\tinma\AppData\Local\Programs\Python\Python310\lib\site-packages\pg8000\core.py", line 658, in execute_simple
    self.handle_messages(context)
  File "c:\Users\tinma\AppData\Local\Programs\Python\Python310\lib\site-packages\pg8000\core.py", line 802, in handle_messages
    code, data_len = ci_unpack(_read(self._sock, 5))
  File "c:\Users\tinma\App

InterfaceError: (pg8000.exceptions.InterfaceError) network error
[SQL:  SELECT * FROM "ETHUSDT" WHERE "Tiempo" > '2023-07-03 20:10:53.744828' ]
(Background on this error at: https://sqlalche.me/e/14/rvf5)

In [ ]:
sleep(15)

if open_order:
    sell_order = client.create_order(
                        symbol=PAIR,
                        side=Client.SIDE_SELL,
                        type=Client.ORDER_TYPE_MARKET,
                        quantity=0.1
                    )
    open_order = False  

In [218]:
sell_order = client.create_order(
                        symbol=PAIR,
                        side=Client.SIDE_SELL,
                        type=Client.ORDER_TYPE_MARKET,
                        quantity=0.1
                    )

In [217]:
precio_entrada

1949.93